# RBE/CS549 Fall 2022: Computer Vision
## Homework 0: Alohomora

Author(s): 
Prof. Nitin J. Sanket (nsanket@wpi.edu), Lening Li (lli4@wpi.edu), Gejji, Vaishnavi Vivek (vgejji@wpi.edu)

Robotics Engineering Department,

Worcester Polytechnic Institute

Code adapted from CMSC733 at the University of Maryland, College Park.


## Phase 1




### Get the BSDS500 dataset

In [ ]:
!wget https://raw.githubusercontent.com/leelening/rbe549/main/hw0/BSDS500.tar.xz
!tar -xvf BSDS500.tar.xz
!mv BSDS500/ /content/data/

In [ ]:
!wget https://raw.githubusercontent.com/leelening/rbe549/main/hw0/TxtFiles.tar.xz
!tar -xvf TxtFiles.tar.xz
!mv TxtFiles/ /content/data/

In [ ]:
import math
import matplotlib.pyplot as plot
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import numpy as np
import cv2
import imutils2

1. Generate Difference of Gaussian Filter Bank: (DoG)
2. Display all the filters in this filter bank and save image as DoG.png,
3. use command "cv2.imwrite(...)"


1. Generate Leung-Malik Filter Bank: (LM)
2. Display all the filters in this filter bank and save image as LM.png,
3. use command "cv2.imwrite(...)"


In [ ]:
"""
Load Images
"""
def load_imgs(folder_name):
	#Storing all the images in the variable imgs
	imgs=[]
	for filename in os.listdir(folder_name):
		img = cv2.imread(os.path.join(folder_name,filename))
		if img is not None:
			imgs.append(img)
		else:
			print("No image found in this folder!!")
	return imgs

In [ ]:
"""
Show Images
"""
def show_img(imgs):

	for img in imgs:
		cv2.namedWindow("Show Image")
		cv2.imshow("Input", img)


In [ ]:

"""
Print Results of a filter Bank in Matplot
"""
def print_filterbank_results_matplot(Filter,file_name, cols):
	i =0
			
	rows = math.ceil(len(Filter )/cols)	
	plot.subplots(rows, cols, figsize=(15,15))
	for index in range(len(Filter )):
		plot.subplot(rows, cols, index+1)
		plot.axis('off')
		plot.imshow(Filter [index], cmap='gray')
	
	plot.savefig(file_name)
	plot.close()

In [ ]:
"""
Chi Square distance
"""
def chisquareDistance(input, bins, filter_bank):

	chi_sq_dis = []
	N = len(filter_bank)
	i = 0
	while i < N:
		left_mask = filter_bank[i]
		right_mask = filter_bank[i+1]		
		tmp = np.zeros(input.shape)
		csd = np.zeros(input.shape)
		min_bin = np.min(input)
	

		for bin in range(bins):
			tmp[input == bin+min_bin] = 1
			g_i = cv2.filter2D(tmp,-1,left_mask)
			h_i = cv2.filter2D(tmp,-1,right_mask)
			term1 = (g_i - h_i)**2
			term2 = 1/(g_i + h_i + np.exp(-7))
			csd += term1*term2

		csd /= 2
		chi_sq_dis.append(csd)
		i = i+2
    	

	return chi_sq_dis


In [ ]:

"""
Generate Gaussian Filter:	
"""
def gaussian(sigma,kernel_size):
	sigma_x, sigma_y = sigma
	Gauss = np.zeros([kernel_size, kernel_size])
	# x = np.linspace(0,kernel_size)
	# y = np.linspace(0,kernel_size)
	if(kernel_size/2):
		index = kernel_size/2
	else:
		index = (kernel_size - 1)/2
	x,y= np.meshgrid(np.linspace(-index,index,kernel_size),np.linspace(-index,index,kernel_size))
	term1 = 0.5/(np.pi*sigma_x*sigma_y)
	
	term2 = np.exp(-((np.square(x)/(np.square(sigma_x))+(np.square(y)/(np.square(sigma_y))))))/2
	Gauss = term1*term2
	return Gauss


In [ ]:



"""
Generate Difference of Gaussian Filter Bank: (DoG)
Display all the filters in this filter bank and save image as DoG.png,
use command "cv2.imwrite(...)"
"""
def DOG_FilterBank(orientation_values,scale_values,kernel_size):

	DOG= []
	"""
	Sobel Filter - Fixed 
	"""

	Sobel_x = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
	Sobel_y = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])


	for scale in scale_values:
		sigma = [scale, scale]
		Gauss = gaussian(sigma,kernel_size)

		DOG_X = cv2.filter2D(Gauss,-1,Sobel_x)
		DOG_Y = cv2.filter2D(Gauss,-1,Sobel_y)
		for ori in range(orientation_values):
			curr_orientation = ori *2 *np.pi /orientation_values
			DOG_Filter = (DOG_X * np.cos(curr_orientation)) +(DOG_Y *np.sin(curr_orientation))
			DOG.append(DOG_Filter)
	# np_array = np.array(DOG, dtype=np.int32)
	# DOG= np_array.astype(np.float32)
	# DOG.astype('float32') 
	# print (DOG)
	fig, axs = plot.subplots(len(scale_values),orientation_values,figsize=(orientation_values,len(scale_values)))
	for i in range(len(scale_values)):
		for j in range(orientation_values):
				
				axs[i, j].imshow(DOG[i*orientation_values+j],cmap='gray')
				axs[i, j].axis('off')

	# plot.show()
	# plot.savefig("/home/uthira/usivaraman_hw0/Phase1/Results/DOG.png")
	plot.close()
	return DOG


In [ ]:

"""
Generate Leung-Malik Filter Bank: (LM)
Display all the filters in this filter bank and save image as LM.png,
use command "cv2.imwrite(...)"
"""
def LM_FilterBank(orientation_values,scale_values,kernel_size):

	LM =[]
	
	"""
	Defining Scales :
	Derivate filter : 3
	Laplacian of Gaussian filter : 8 (sigma+sigma*3)
	Gaussian : 4
	"""
	Derivatives_scale = scale_values[0:3]
	GaussinaFilter_scale = scale_values
	LOGFilter_scale =[]
	for i in range(len(scale_values)):
		scale = scale_values[i]
		LOG_scale_value = scale + 3 *scale
		LOGFilter_scale.append(LOG_scale_value)

	"""
		First and Second Derivatives of Gauss Filter
	"""
	FirstD =[]
	SecondD=[]
	for scale in Derivatives_scale :
		sigma = [scale, scale]

		
		del_x = np.array([[-1, 0, 1],[-2, 0, 2], [-1, 0, 1]])
		del_y = np.array([[-1, -2, -1],[0, 0, 0], [1, 2, 1]])
		Gauss = gaussian(sigma,kernel_size)		
		FirstD_x = cv2.filter2D(Gauss,-1,del_x)
		FirstD_y = cv2.filter2D(Gauss,-1,del_y)
		SecondD_x = cv2.filter2D(FirstD_x,-1,del_x)
		SecondD_Y = cv2.filter2D(FirstD_y,-1,del_y)
		for ori in range(orientation_values):
			curr_orientation = ori *2 *np.pi /orientation_values
			FirstD_Filter = (FirstD_x* np.cos(curr_orientation)) +(FirstD_y *np.sin(curr_orientation))
			SecondtD_Filter = (SecondD_x* np.cos(curr_orientation)) +(SecondD_Y *np.sin(curr_orientation))
			FirstD.append(FirstD_Filter)
			SecondD.append(SecondtD_Filter)

	"""
	Laplacian of Gaussian Filter
	
	"""
	LOG=[]
	if(kernel_size/2):
			index = kernel_size/2
	else:
			index = (kernel_size - 1)/2
	for scale in LOGFilter_scale:
		sigma = [scale, scale]
		Gauss = gaussian(sigma, kernel_size)
		Log_kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
		LOG_Filter= cv2.filter2D(Gauss,-1,Log_kernel)
		LOG.append(LOG_Filter)
		# x,y= np.meshgrid(np.linspace(-index,index,kernel_size),np.linspace(-index,index,kernel_size))
		# term1 = -1/(np.pi*np.square(sigma[0])*np.square(sigma[1]))
		# term2 = (1-((np.square(x)/(np.square(sigma[0]))+(np.square(y)/(np.square(sigma[1]))))))
		# term3 = np.exp(-((np.square(x)/(np.square(sigma[0]))+(np.square(y)/(np.square(sigma[1]))))))/2
		# LOG_filter = term1*term2*term3
		# LOG.append(LOG_filter)

	"""
	Gaussian Filter
	"""
	Gaussian=[]
	for scale in GaussinaFilter_scale:
		sigma = [scale, scale]
		Gaussian.append(gaussian(sigma, kernel_size))
	
	LM = FirstD +SecondD + LOG + Gaussian

	fig, axs = plot.subplots(len(scale_values),orientation_values,figsize=(orientation_values,len(scale_values)))
	for i in range(len(scale_values)):
		for j in range(orientation_values):
				
				axs[i, j].imshow(LM[i*orientation_values+j],cmap='gray')
				axs[i, j].axis('off')
	# plot.show()
	return LM


In [ ]:

"""
Generate Sine Wave:	
"""
def sinewave(frequency,kernel_size,angle):
	
	
	index = (kernel_size - 1)/2

	x, y = np.meshgrid(np.linspace(-index, index+1, kernel_size), np.linspace(-index, index+1, kernel_size))
	value= x * np.cos(angle) + y * np.sin(angle)
	sin2d = np.sin(value* 2 * np.pi * frequency/kernel_size)

	return sin2d



"""
Generate Gabor Filter Bank: (Gabor)
Display all the filters in this filter bank and save image as Gabor.png,
use command "cv2.imwrite(...)"
"""
def Gabor_FilterBank(orientation_values,scale_values,frequency_values,kernel_size):

	Gabor =[]
	for scale in scale_values:
		sigma = [scale, scale]
		Gauss = gaussian(sigma,kernel_size)
		for angle in range(orientation_values):
			for frequency in frequency_values:
				SinewaveFilter =sinewave(frequency,kernel_size,angle)
				Gabor_Filter = Gauss*SinewaveFilter
				Gabor.append(Gabor_Filter)
	fig, axs = plot.subplots(len(scale_values),orientation_values,figsize=(orientation_values,len(scale_values)))
	for i in range(len(scale_values)):
		for j in range(orientation_values):
				
				axs[i, j].imshow(Gabor[i*orientation_values+j],cmap='gray')
				axs[i, j].axis('off')
	# plot.show()
	# plot.savefig("/home/uthira/usivaraman_hw0/Phase1/Results/Gabor.png")
	plot.close()
	return Gabor


In [ ]:


"""
Generate Half-disk masks
Display all the Half-disk masks and save image as HDMasks.png,
use command "cv2.imwrite(...)"
"""
def HalfDisk(radius, angle):
	# radius = 3
	# Half_disk_masks =np.ones((8,8))
	# y,x = np.ogrid[-3: 3+1, 0: 3+1]
	# Half_disk_masks = x**2+y**2 <= 3**2
	# Half_disk_masks = np.array(Half_disk_masks)
	# print(Half_disk_masks)
	# print_filterbank_results_matplot(Half_disk_masks,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/HalfDiskMasks.png', 8)
	size = 2*radius + 1
	centre = radius
	half_disk = np.zeros([size, size])
	for i in range(radius):
		for j in range(size):
			distance = np.square(i-centre) + np.square(j-centre)
			if distance <= np.square(radius):
				half_disk[i,j] = 1
    
	
	half_disk = imutils.rotate(half_disk, angle)
	half_disk[half_disk<=0.5] = 0
	half_disk[half_disk>0.5] = 1
	return half_disk
def halfdiskFilters(radii, orientations):
	filter_bank = []
	for radius in radii:
		filter_bank_pairs = []
		temp = []
		for orientation in range(orientations):
			angle = orientation * 360 / orientations
			half_disk_filter = HalfDisk(radius, angle)
			temp.append(half_disk_filter)

        #to make pairs
		i = 0
		while i < orientations/2:
			filter_bank_pairs.append(temp[i])
			filter_bank_pairs.append(temp[i+int((orientations)/2)])
			i = i+1

		filter_bank+=filter_bank_pairs
	
	
	return filter_bank	

In [ ]:

def pblite_edges(T_g, B_g, C_g, Canny_edge, Sobel_edges, weights):
	Canny_edge = cv2.cvtColor(Canny_edge, cv2.COLOR_BGR2GRAY)
	Sobel_edges = cv2.cvtColor(Sobel_edges, cv2.COLOR_BGR2GRAY)
	T1 = (T_g + B_g + C_g)/3
	w1 = weights[0]
	w2 = weights[1]
	T2 = (w1 * Canny_edge) + (w2 * Sobel_edges)
	# print(T1.shape)
	# print(T2.shape)
	pb_lite_output = np.multiply(T1, T2)
	return pb_lite_output



In [ ]:




"""Derivative of Gaussian Filter"""	
DOG_FB = DOG_FilterBank(18,[3,3],36)		
# print_filterbank_results_matplot(DOG_FB,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/DOG.png',8)



"""LM Filter"""	
LM_FB_small = LM_FilterBank(6,[1,np.sqrt(2),2,2*np.sqrt(2)],49)		
LM_FB_large= LM_FilterBank(6,[np.sqrt(2),2,2*np.sqrt(2),4],49)	
# print_filterbank_results_matplot(LM_FB_small,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/LM_small.png',12)
# print_filterbank_results_matplot(LM_FB_large,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/LM_large.png',12)
LM_FB = LM_FB_small + LM_FB_large
# print_filterbank_results_matplot(LM_FB,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/LM.png',12)

fig, axs = plot.subplots(8,6,figsize=(6,8))
for i in range(8):
	for j in range(6):
			
			axs[i, j].imshow(LM_FB[i*6+j],cmap='gray')
			axs[i, j].axis('off')
# plot.show()


"""Gabor Filter"""	
Gabor_FB = Gabor_FilterBank(6,[20,45],[3,4,6],49)		
# print_filterbank_results_matplot(Gabor_FB,'/home/uthira/CV/Cvis_HW0/Phase1/Results/Filter/Gabor.png',8)


"""
Total Filter Bank 
"""


textron_gradients = []
brightness_gradients = []
color_gradients = []
half_disk_filter_bank = halfdiskFilters([2,5,10,20,30], 16)
index =0

images_folder = "./content/data/BSDS500/Images/"
results_folder = "./Results/"

image_files = os.listdir(images_folder)
for img_name in image_files:
	print("image name",img_name)
	img_path = images_folder + img_name
	img = cv2.imread(img_path)
	# plot.imshow(img, cmap = "gray")
	# plot.show()
	print("index:",index)
	image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	filterbank = DOG_FB + LM_FB + Gabor_FB
	filtered_images =[]
	for fb in filterbank:
		Results = cv2.filter2D(image_gray,-1, fb)	
		filtered_images.append(Results)
	filename = results_folder+"Filter/"+"Fbank_"+img_name
	# cv2.imwrite(os.path.join(path , image_name), img)
# cv2.waitKey(0)
	# print_filterbank_results_matplot(filtered_images,filename,8)		
	# cv2.imshow("output", Results)
	# cv2.waitKey()
	
	"""
	Generate Texton Map
	Filter image using oriented gaussian filter bank
	"""
	filtered_images= np.array(filtered_images)
	# print("Shape of Filtered Images")
	# print(filtered_images.shape)
	# print("Length of Filterbank")
	# print(len(filterbank))
	f,x,y = filtered_images.shape
	input_mat = filtered_images.reshape([f, x*y])
	input_mat = input_mat.transpose()
	# print("Input to Kmeans clustering")
	# print(input_mat)
	# print(input_mat.shape)
	



	"""
	Generate texture ID's using K-means clustering
	Display texton map and save image as TextonMap_ImageName.png,
	use command "cv2.imwrite('...)"
	"""
	print("Textron Maps")
	kmeans = KMeans(n_clusters=64, init='k-means++', max_iter=300, n_init=2, random_state=0)
	kmeans.fit(input_mat)
	labels = kmeans.predict(input_mat)
	texton_image = labels.reshape([x,y])
	print(texton_image.shape)
	image_name ="Texton_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), T_g)
	plot.imsave(os.path.join(results_folder , image_name),  texton_image, cmap = None)
	# textron_maps.append(texton_image)
	"""
	Generate Texton Gradient (Tg)
	Perform Chi-square calculation on Texton Map
	Display Tg and save image as Tg_ImageName.png,
	use command "cv2.imwrite(...)"
	"""
	T_g = chisquareDistance(texton_image, 64, half_disk_filter_bank)
	T_g = np.array(T_g)
	T_g = np.mean(T_g, axis = 0)
	#plt.imshow(T_g)
	#plt.show()	
	textron_gradients.append(T_g)
	# plot.imshow(T_g, cmap=None)
	# plot.show()	
	# plot.close()
	# T_g = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
	image_name ="Tg_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), T_g)
	plot.imsave(os.path.join(results_folder , image_name),  T_g, cmap = None)

	# plot.imshow(texton_image)	
	# plot.show()	
	# plot.close()
	# plot.imsave(os.path.join("/home/uthira/CV/Cvis_HW0/Phase1/Results/Textron_map/", "TextonMap_"+str(index)) , texton_image)
	

	"""
	Generate Brightness Map
	Perform brightness binning 
	"""
	print("generating brightness maps..")
	
	# for i,image in enumerate(images):
	image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	x,y = image_gray.shape
	input_mat = image_gray.reshape([x*y,1])
	kmeans = KMeans(n_clusters = 16, n_init = 4)
	kmeans.fit(input_mat)
	labels = kmeans.predict(input_mat)
	brightness_image = labels.reshape([x,y])
	print(brightness_image.shape)
	image_name ="Bright_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), T_g)
	plot.imsave(os.path.join(results_folder , image_name),  brightness_image, cmap = None)
	"""
	Generate Brightness Gradient (Bg)
	Perform Chi-square calculation on Brightness Map
	Display Bg and save image as Bg_ImageName.png,
	use command "cv2.imwrite(...)"
	"""
	# brightness_maps.append(brightness_image)
	B_g = chisquareDistance(brightness_image, 16, half_disk_filter_bank)
	B_g = np.array(B_g)
	B_g = np.mean(B_g, axis = 0)
	#plt.imshow(B_g)
	#plt.show()
	brightness_gradients.append(B_g)
	image_name ="Bg_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), B_g)
	plot.imsave(os.path.join(results_folder , image_name),  B_g, cmap = None)
		# plot.imshow(brightness_image)
		# plot.show()
		# plt.imsave(folder_name + "results/Brightness_map/BrightnessMap_" + file_names[i], brightness_image)

	

	"""
	Generate Color Map
	Perform color binning or clustering
	"""
	print("generating color maps..")
	
	
	x,y,c = img.shape
	input_mat = img.reshape([x*y,c])

	kmeans = KMeans(n_clusters =16, n_init = 4)
	kmeans.fit(input_mat)
	labels = kmeans.predict(input_mat)
	color_image = labels.reshape([x,y])
	image_name ="Color_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), T_g)
	plot.imsave(os.path.join(results_folder , image_name),  color_image, cmap = None)
	# color_maps.append(color_image)
	"""
	Generate Color Gradient (Cg)
	Perform Chi-square calculation on Color Map
	Display Cg and save image as Cg_ImageName.png,
	use command "cv2.imwrite(...)"
	"""
	C_g = chisquareDistance(color_image,16, half_disk_filter_bank)
	C_g = np.array(C_g)
	C_g = np.mean(C_g, axis = 0)
	# print(color_image.shape)
	#plt.imshow(C_g)
	#plt.show()
	color_gradients.append(C_g)
	image_name ="Cg_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), C_g)
	plot.imsave(os.path.join(results_folder , image_name),  C_g, cmap = None)
		#plt.imshow(color_image)
		#plt.show()			
		# plt.imsave(folder_name + "results/Color_map/ColorMap_"+ file_names[i], color_image)
	"""
	Combine responses to get pb-lite output
	Display PbLite and save image as PbLite_ImageName.png
	use command "cv2.imwrite(...)"
	"""
	print("generating pb lite output..")
	# for i in range(len(images)):	
	sobel_pb = cv2.imread("./content/data/BSDS500/SobelBaseline/" + img_name)
	canny_pb = cv2.imread("./content/data/BSDS500/SobelBaseline/" + img_name)
	pb_edge = pblite_edges(T_g, B_g, C_g, canny_pb, sobel_pb, [0.5,0.5])
	print("Final Output")
	# plot.imshow(pb_edge, cmap = "gray")
	plot.show()
	# pb_edge =cv2.cvtColor(pb_edge, cv2.COLOR_BGR2GRAY)
	image_name ="PbLite_"+img_name
	# cv2.imwrite(os.path.join(results_folder , image_name), pb_edge)
	plot.imsave(os.path.join(results_folder , image_name), pb_edge, cmap = "gray")
	# /home/uthira/CV/Cvis_HW0/Phase1/Results/Pblite/Figure_1.png
	
	index = index +1



## Phase 2

### Neural Network Construction

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def loss_fn(out, labels):
    ###############################################
    # Fill your loss function of choice here!
    ###############################################
    LossFn = nn.CrossEntropyLoss()
    loss = LossFn(out, labels)
    return loss

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = loss_fn(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = loss_fn(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'loss': loss.detach(), 'acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'loss': epoch_loss.item(), 'acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, result['loss'], result['acc']))


##################################
#Basic
##################################  

class CIFAR10Model(ImageClassificationBase):
    def __init__(self, InputSize, OutputSize):
    
        super().__init__()
        

        """
        Inputs: 
        InputSize - Size of the Input
        OutputSize - Size of the Output
        """
        #############################
        # Fill your network initialization of choice here!
            # Fill your network initialization of choice here!
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),nn.MaxPool2d(kernel_size = 10, stride = 10))   
        self.fc = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU())
        self.fc1= nn.Sequential(
            nn.Linear(512, OutputSize))

    
    #############################
    
    
    

      
    def forward(self, xb):
        """
        Input:
        xb is a MiniBatch of the current image
        Outputs:
        out - output of the network
        """
        #############################
        # Fill your network structure of choice here!
        #############################
        out = self.layer1(xb)
        out = self.layer2(out)
        out = self.layer3(out)
        # out = self.layer4(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        out = self.fc1(out)
        out = F.softmax(out)
        return out


##################################
#ResNet 18
##################################    

class ResNet(ImageClassificationBase):
    def __init__(self,OutputSize):
        super().__init__()
        
        self.conv_1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(64),
                                nn.ReLU(),
                                nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(128),
                                nn.ReLU(),
                                nn.MaxPool2d(2))

        self.res_1 = nn.Sequential(nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(128),
                                nn.ReLU(),
                                nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(128),
                                nn.ReLU())

        self.conv_2 = nn.Sequential(nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(256),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(512),
                                nn.ReLU(),
                                nn.MaxPool2d(2))

        self.res_2 = nn.Sequential(nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(512),
                                nn.ReLU(),
                                nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
                                nn.BatchNorm2d(512),
                                nn.ReLU())
        
        self.fc = nn.Sequential(  
                                        nn.Linear(512, OutputSize))
        
    def forward(self, xb):
        out = self.conv_1(xb)
        # print(out.shape)
        out = self.res_1(out) + out
        # print(out.shape)
        out = self.conv_2(out)
        # print(out.shape)
        out = self.res_2(out) + out
        # print(out.shape)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


##################################
#ResNet 34
##################################        
# class ResNet(ImageClassificationBase):
    # def __init__(self, OutputSize):
    #     super().__init__()
    #     #34 layer res_net
    #     self.conv_1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(64),
    #                             nn.ReLU(),
    #                             nn.Conv2d(64, 64, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(64),
    #                             nn.ReLU(),
    #                             nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(128),
    #                             nn.ReLU(),
    #                             nn.MaxPool2d(2))

    #     self.res_1 = nn.Sequential(nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(128),
    #                             nn.ReLU(),
    #                             nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(128),
    #                             nn.ReLU(),
    #                             nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(128),
    #                             nn.ReLU(),
    #                             nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(128),
    #                             nn.ReLU())

    #     self.conv_2 = nn.Sequential(nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(256),
    #                             nn.ReLU(),
    #                             nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(256),
    #                             nn.ReLU(),
    #                             nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(256),
    #                             nn.ReLU(),
    #                             # nn.MaxPool2d(2),
    #                             nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(256),
    #                             nn.ReLU(),
    #                             nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(256),
    #                             nn.ReLU(),
    #                             nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(512),
    #                             nn.ReLU())

    #     self.res_2 = nn.Sequential(nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(512),
    #                             nn.ReLU(),
    #                             nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(512),
    #                             nn.ReLU(),
    #                             nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
    #                             nn.BatchNorm2d(512),
    #                             nn.ReLU())
    #     self.avgpool = nn.AvgPool2d(5)
    #     self.fc= nn.Sequential(nn.Linear(512, OutputSize)) 
        
    # def forward(self, xb):
    #     out = self.conv_1(xb)
    #     # print(out.shape)
    #     out = self.res_1(out) + out
    #     # print(out.shape)
    #     out = self.conv_2(out)
    #     # print(out.shape)
    #     out = self.res_2(out) + out
    #     out = self.avgpool(out)
    #     # print(out.shape)
    #     out = torch.flatten(out, 1)
    #     # print(out.shape)
    #     out = self.fc(out)
    #     # print(out.shape)
    #     return out

##################################
#DenseNet
################################## 
class block(nn.Module):
    def __init__(self, in_channels,growth_rate,bottleneck_width):
        super().__init__()
        # print("Inchannels in block :",in_channels)
        self.layers = nn.Sequential(
                                    nn.BatchNorm2d(in_channels),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels, bottleneck_width*growth_rate, kernel_size=1, padding=0),          #1x1 convolution
                                    nn.BatchNorm2d(bottleneck_width*growth_rate),
                                    nn.ReLU(),
                                    nn.Conv2d(bottleneck_width*growth_rate, growth_rate, kernel_size=3, padding=1))                    #3x3 convolution
              

    def forward(self, xb):
        
        out = self.layers(xb)        
        out = torch.cat([xb, out], 1)            
        return out


class DenseNet(ImageClassificationBase):

    def __init__(self):
        super().__init__()
        growth_rate = 6                                                                
        num_layers= 6                                                           
        num_features= 12
        bottleneck_width = 4
        self.conv1 = nn.Conv2d(3, 12, kernel_size = 1)  
                      
        self.dense1 = self.dense(num_features,growth_rate, num_layers,bottleneck_width)
        num_features += num_layers * growth_rate  
        # print(num_features)
        self.trans1 = nn.Sequential(nn.BatchNorm2d(num_features),                    
                                        nn.Conv2d(num_features, 48, kernel_size=1, padding=0),
                                        nn.AvgPool2d(2))
        num_features= 48
        self.dense2 = self.dense(num_features, growth_rate, num_layers,bottleneck_width)
        num_features += num_layers * growth_rate 
        print(num_features)
        self.trans2 = nn.Sequential(nn.BatchNorm2d(num_features),
                                        nn.ReLU(),
                                        nn.Conv2d(num_features, 96, kernel_size=1, padding=0)
                                        )

        num_features= 96
        self.dense3 = self.dense(num_features,growth_rate, num_layers,bottleneck_width)
        num_features += num_layers * growth_rate
        print(num_features)
        self.classifier = nn.Sequential(nn.BatchNorm2d(num_features),            
                                        nn.AvgPool2d(5),
                                        nn.Flatten(),
                                        nn.Linear(132, 10))

    def dense(self, features, grate, num_layers,bottleneck_width):                            
        layers = []
        for i in range(num_layers):
            layers.append(block(features, grate,bottleneck_width))
            features += grate
            # print("features :",features)
        # print(layers)
        return nn.Sequential(*layers)

    def forward(self, xb):
        out = self.conv1(xb)
        # print(out.shape)
        out = self.dense1(out)
        # print(out.shape)
        out = self.trans1(out)
        # print(out.shape)
        out = self.dense2(out)
        # print(out.shape)
        out = self.trans2(out)
        # print(out.shape)
        out = self.dense3(out)
        # out = torch.flatten(out, 1)
        # print(out.shape)
        out = self.classifier(out)
        # print(out.shape)

        return out


##################################
#ResNeXT
##################################

class res_block(nn.Module):
    def __init__(self, in_features, num_layers,inplanes, stride):
        super().__init__()
        features = num_layers*inplanes             
        self.layers = nn.Sequential(nn.Conv2d(in_features, features, kernel_size = 1),
                                    nn.BatchNorm2d(features),
                                    nn.Conv2d(features, features, kernel_size = 3, stride = stride, padding = 1, groups = num_layers), #2 groups made(as C=2 in our case)
                                    nn.BatchNorm2d(features),
                                    nn.Conv2d(features, 2*features, kernel_size = 1),
                                    nn.BatchNorm2d(2*features))
        self.residual = nn.Sequential(nn.Conv2d(in_features, 2*features, kernel_size = 1, stride = stride),
                                    nn.BatchNorm2d(2*features))
    
    def forward(self, xb):
        out = self.layers(xb)
        out += self.residual(xb)
        out = F.relu(out)
        return out

class ResNeXt(ImageClassificationBase):

    def __init__(self):
        super().__init__()
        self.in_channels = 64
       
        self.num_layers = 2
        self.in_features = 64
        self.bottleneck_width = 2

        self.conv1 = nn.Conv2d(3, 64, kernel_size = 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.block1 = self.block(self.in_channels, self.num_layers, self.in_features, 1)
        self.in_channels = self.bottleneck_width*self.num_layers*self.in_features
        self.in_features = self.in_features*2
        self.block2 = self.block(self.in_channels, self.num_layers, self.in_features,2) 
        self.in_channels = self.bottleneck_width*self.num_layers*self.in_features
        self.in_features = self.in_features*2          
        self.block3 = self.block(self.in_channels, self.num_layers, self.in_features,2)
        
        self.avgpool = nn.AvgPool2d(8)
        # self.batchnorm=nn.BatchNorm2d(1024)
        self.fc = nn.Sequential(nn.Linear(1024,10))
        # self.fc1 =nn.Sequential( nn.Linear(9126,10))                               



    def block(self,in_,nlayers,inf, stride):
        layers = []
        layers.append(res_block(in_, nlayers, inf, stride))        
        return nn.Sequential(*layers)
        
    def forward(self,xb):
        out = self.conv1(xb)
        # print(out.shape)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.block1(out)
        # print(out.shape)
        out = self.block2(out)
        # print(out.shape)
        out = self.block3(out)
        out= self.avgpool(out)
        # out= self.batchnorm(out)
        out = torch.flatten(out, 1) 
        
        # print(out.shape)
        out = self.fc(out)

        return out

      

In [ ]:
import time
import glob
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

def tic():
    """
    Function to start timer
    Tries to mimic tic() toc() in MATLAB
    """
    StartTime = time.time()
    return StartTime

def toc(StartTime):
    """
    Function to stop timer
    Tries to mimic tic() toc() in MATLAB
    """
    return time.time() - StartTime

def FindLatestModel(CheckPointPath):
    """
    Finds Latest Model in CheckPointPath
    Inputs:
    CheckPointPath - Path where you have stored checkpoints
    Outputs:
    LatestFile - File Name of the latest checkpoint
    """
    FileList = glob.glob(CheckPointPath + '*.ckpt.index') # * means all if need specific format then *.csv
    LatestFile = max(FileList, key=os.path.getctime)
    # Strip everything else except needed information
    LatestFile = LatestFile.replace(CheckPointPath, '')
    LatestFile = LatestFile.replace('.ckpt.index', '')
    return LatestFile


def convertToOneHot(vector, NumClasses):
    """
    Inputs:
    vector - vector of argmax indexes
    NumClasses - Number of classes
    """
    return np.equal.outer(vector, np.arange(NumClasses)).astype(np.float)

### Train your neural network

In [ ]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision import transforms as tf
from torch.optim import Adam
from torchvision.datasets import CIFAR10

import cv2
import sys
import os
import numpy as np
import random
import skimage
import PIL
import os
import glob
import random
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import time
from torchvision.transforms import ToTensor
import argparse
import shutil
import string
from termcolor import colored, cprint
import math as m
from tqdm.notebook import tqdm

# Don't generate pyc codes
sys.dont_write_bytecode = True
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

def SetupAll(CheckPointPath):
    """
    Inputs: 
    CheckPointPath - Path to save checkpoints/model
    Outputs:
    SaveCheckPoint - Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    ImageSize - Size of the image
    NumTrainSamples - length(Train)
    TrainLabels - Labels corresponding to Train
    NumClasses - Number of classes
    """
    # Read and Setup Labels
    LabelsPathTrain = '/content/data/TxtFiles/LabelsTrain.txt'
    TrainLabels = ReadLabels(LabelsPathTrain)

    # If CheckPointPath doesn't exist make the path
    if(not (os.path.isdir(CheckPointPath))):
       os.makedirs(CheckPointPath)
        
    # Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    SaveCheckPoint = 100 
    
    # Image Input Shape
    ImageSize = [32, 32, 3]
    NumTrainSamples = len(TrainSet)

    # Number of classes
    NumClasses = 10

    return SaveCheckPoint, ImageSize, NumTrainSamples, TrainLabels, NumClasses


def ReadLabels(LabelsPathTrain):
    if(not (os.path.isfile(LabelsPathTrain))):
        print('ERROR: Train Labels do not exist in '+LabelsPathTrain)
        sys.exit()
    else:
        TrainLabels = open(LabelsPathTrain, 'r')
        TrainLabels = TrainLabels.read()
        TrainLabels = map(float, TrainLabels.split())

    return TrainLabels
    

def ReadDirNames(ReadPath):
    """
    Inputs: 
    ReadPath is the path of the file you want to read
    Outputs:
    DirNames is the data loaded from /content/data/TxtFiles/DirNames.txt which has full path to all image files without extension
    """
    # Read text files
    DirNames = open(ReadPath, 'r')
    DirNames = DirNames.read()
    DirNames = DirNames.split()
    return DirNames

    
def GenerateBatch(TrainSet, TrainLabels, ImageSize, MiniBatchSize):
    """
    Inputs: 
    TrainSet - Variable with Subfolder paths to train files
    NOTE that Train can be replaced by Val/Test for generating batch corresponding to validation (held-out testing in this case)/testing
    TrainLabels - Labels corresponding to Train
    NOTE that TrainLabels can be replaced by Val/TestLabels for generating batch corresponding to validation (held-out testing in this case)/testing
    ImageSize is the Size of the Image
    MiniBatchSize is the size of the MiniBatch
   
    Outputs:
    I1Batch - Batch of images
    LabelBatch - Batch of one-hot encoded labels 
    """
    I1Batch = []
    LabelBatch = []
    
    ImageNum = 0
    while ImageNum < MiniBatchSize:
        # Generate random image
        RandIdx = random.randint(0, len(TrainSet)-1)
        
        ImageNum += 1
    	
    	  ##########################################################
    	  # Add any standardization or data augmentation here!
    	  ##########################################################

        I1, Label = TrainSet[RandIdx]

        # Append All Images and Mask
        I1Batch.append(I1)
        LabelBatch.append(torch.tensor(Label))
        
    return torch.stack(I1Batch), torch.stack(LabelBatch)


def PrettyPrint(NumEpochs, DivTrain, MiniBatchSize, NumTrainSamples, LatestFile):
    """
    Prints all stats with all arguments
    """
    print('Number of Epochs Training will run for ' + str(NumEpochs))
    print('Factor of reduction in training data is ' + str(DivTrain))
    print('Mini Batch Size ' + str(MiniBatchSize))
    print('Number of Training Images ' + str(NumTrainSamples))
    if LatestFile is not None:
        print('Loading latest checkpoint with the name ' + LatestFile)              

def TrainOperation(TrainLabels, NumTrainSamples, ImageSize,
                   NumEpochs, MiniBatchSize, SaveCheckPoint, CheckPointPath,
                   DivTrain, LatestFile, TrainSet, LogsPath):
    """
    Inputs: 
    TrainLabels - Labels corresponding to Train/Test
    NumTrainSamples - length(Train)
    ImageSize - Size of the image
    NumEpochs - Number of passes through the Train data
    MiniBatchSize is the size of the MiniBatch
    SaveCheckPoint - Save checkpoint every SaveCheckPoint iteration in every epoch, checkpoint saved automatically after every epoch
    CheckPointPath - Path to save checkpoints/model
    DivTrain - Divide the data by this number for Epoch calculation, use if you have a lot of dataor for debugging code
    LatestFile - Latest checkpointfile to continue training
    TrainSet - The training dataset
    LogsPath - Path to save Tensorboard Logs
    Outputs:
    Saves Trained network in CheckPointPath and Logs to LogsPath
    """
   # Initialize the model
    # model = CIFAR10Model(InputSize=3*32*32,OutputSize=10) 
    
    # model = ResNet(10)
    model = ResNeXt()
    # model = DenseNet()
    model = model.to(device)
    ###############################################
    # Fill your optimizer of choice here!
    ###############################################
    Optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

    # Tensorboard
    # Create a summary to monitor loss tensor
    Writer = SummaryWriter(LogsPath)

    if LatestFile is not None:
        CheckPoint = torch.load(CheckPointPath + LatestFile + '.ckpt')
        # Extract only numbers from the name
        StartEpoch = int(''.join(c for c in LatestFile.split('a')[0] if c.isdigit()))
        model.load_state_dict(CheckPoint['model_state_dict'])
        print('Loaded latest checkpoint with the name ' + LatestFile + '....')
    else:
        StartEpoch = 0
        print('New model initialized....')
    train = []
    for Epochs in tqdm(range(StartEpoch, NumEpochs)):
        NumIterationsPerEpoch = int(NumTrainSamples/MiniBatchSize/DivTrain)
        Batches=[]
        for PerEpochCounter in tqdm(range(NumIterationsPerEpoch)):
            # print("Epoch Counter :",PerEpochCounter)
            Batch = GenerateBatch(TrainSet, TrainLabels, ImageSize, MiniBatchSize)
            
            # Predict output with forward pass
            LossThisBatch = model.training_step(Batch)

            Optimizer.zero_grad()
            LossThisBatch.backward()
            Optimizer.step()
            
            # Save checkpoint every some SaveCheckPoint's iterations
            if PerEpochCounter % SaveCheckPoint == 0:
                # Save the Model learnt in this epoch
                SaveName =  CheckPointPath + str(Epochs) + 'a' + str(PerEpochCounter) + 'model.ckpt'
                
                torch.save({'epoch': Epochs,'model_state_dict': model.state_dict(),'optimizer_state_dict': Optimizer.state_dict(),'loss': LossThisBatch}, SaveName)
                # print('\n' + SaveName + ' Model Saved...')
            Batches.append(Batch)
            # result = model.validation_step(Batch)
            # model.epoch_end(Epochs*NumIterationsPerEpoch + PerEpochCounter, result)
        model.eval()
        outputs = [model.validation_step(batch) for batch in Batches]
        train_result = model.validation_epoch_end(outputs)
        model.epoch_end(Epochs, train_result)
        train.append(train_result)
        # Tensorboard
        Writer.add_scalar('LossEveryEpoch', train_result["loss"], Epochs)
        Writer.add_scalar('Accuracy', train_result["acc"], Epochs)
        # If you don't flush the tensorboard doesn't update until a lot of iterations!
        Writer.flush()
        

        # Save model every epoch
        SaveName = CheckPointPath + str(Epochs) + 'model.ckpt'
        torch.save({'epoch': Epochs,'model_state_dict': model.state_dict(),'optimizer_state_dict': Optimizer.state_dict(),'loss': LossThisBatch}, SaveName)
        print('\n' + SaveName + ' Model Saved...')

    return train
def plot_losses(history):
    losses = [x['loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Train Loss vs. No. of epochs')
    # plt.show()  
    plt.savefig('/home/usivaraman/CV/Cvis_HW0/Phase2/Code/Results/JAN17/ResNeXt_Loss.png')

def plot_accuracies(history):
    accuracies = [x['acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title(' Train Accuracy vs. No. of epochs')
    # plt.show()   
    plt.savefig('/home/usivaraman/CV/Cvis_HW0/Phase2/Code/Results/JAN17/ResNeXt_Acc.png') 
        


# Default Hyperparameters
NumEpochs = 50
# transforms = tf.Compose([tf.CenterCrop(10), tf.ToTensor(),tf.RandomRotation((30,70)),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # DenseNet,ResNet (18 layer), ResNet (34 layer)
transforms = tf.Compose([ tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # ResNext
TrainSet = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=tf.ToTensor())
DivTrain = 1.0
MiniBatchSize = 256
LoadCheckPoint = 0
CheckPointPath = "/content/Checkpoints/"
LogsPath = "/content/Logs"

# Setup all needed parameters including file reading
SaveCheckPoint, ImageSize, NumTrainSamples, TrainLabels, NumClasses = SetupAll(CheckPointPath)

# Find Latest Checkpoint File
if LoadCheckPoint==1:
    LatestFile = FindLatestModel(CheckPointPath)
else:
    LatestFile = None

# Pretty print stats
PrettyPrint(NumEpochs, DivTrain, MiniBatchSize, NumTrainSamples, LatestFile)

train = TrainOperation(TrainLabels, NumTrainSamples, ImageSize,
                NumEpochs, MiniBatchSize, SaveCheckPoint, CheckPointPath,
                DivTrain, LatestFile, TrainSet, LogsPath)
    
plot_accuracies(train)
plot_losses(train)

### Test your neural network

In [ ]:
import cv2
import os
import sys
import glob
import random
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import numpy as np
import time
from torchvision import datasets
from torchvision import transforms as tf
from torch.optim import Adam
from torchvision.transforms import ToTensor
import argparse
import shutil
import string
import math as m
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
import torch

# import PrettyTable
import seaborn as sns
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
# Don't generate pyc codes
sys.dont_write_bytecode = True

def SetupAll():
    """
    Outputs:
    ImageSize - Size of the Image
    """   
    # Image Input Shape
    ImageSize = [32, 32, 3]

    return ImageSize

def StandardizeInputs(Img):
    ##########################################################################
    # Add any standardization or cropping/resizing if used in Training here!
    ##########################################################################
    return Img
    
def ReadImages(Img):
    """
    Outputs:
    I1Combined - I1 image after any standardization and/or cropping/resizing to ImageSize
    I1 - Original I1 image for visualization purposes only
    """    
    I1 = Img
    
    if(I1 is None):
        # OpenCV returns empty list if image is not read! 
        print('ERROR: Image I1 cannot be read')
        sys.exit()
        
    I1S = StandardizeInputs(np.float32(I1))

    I1Combined = np.expand_dims(I1S, axis=0)

    return I1Combined, I1
                

def TestOperation(ImageSize, ModelPath, TestSet, LabelsPathPred):
    """
    Inputs: 
    ImageSize is the size of the image
    ModelPath - Path to load trained model from
    TestSet - The test dataset
    LabelsPathPred - Path to save predictions
    Outputs:
    Predictions written to /content/data/TxtFiles/PredOut.txt
    
    """
    # Predict output with forward pass, MiniBatchSize for Test is 1
    model = CIFAR10Model(InputSize=3*32*32,OutputSize=10) 
    # model = DenseNet()
    # model = ResNet(10)
    # model = ResNeXt()
    model = model.to(device)
    print(model)
    
    CheckPoint = torch.load(ModelPath)
    model.load_state_dict(CheckPoint['model_state_dict'])
    
  
    print('Number of parameters in this model are %d ' % len(model.state_dict().items()))
    
    OutSaveT = open(LabelsPathPred, 'w')
    label_set=[]
    pred_set =[]
    for count in tqdm(range(len(TestSet))): 
        Img, Label = TestSet[count]
        # print(Img.shape)
        Img, ImgOrg = ReadImages(Img)
        # print(Img)
       
        Img = torch.from_numpy(Img)
        # Img = Img.unsqueeze(0)
        # print(Img.shape)
        # plt.imshow(ImgOrg)
        # Img = torch.from_numpy(Img)
        model.eval()
        PredT = torch.argmax(model(Img)).item()
        label_set.append(Label)
        pred_set.append(PredT)
        OutSaveT.write(str(PredT)+'\n')
    OutSaveT.close()
    return label_set,pred_set

def Accuracy(Pred, GT):
    """
    Inputs: 
    Pred are the predicted labels
    GT are the ground truth labels
    Outputs:
    Accuracy in percentage
    """
    return (np.sum(np.array(Pred)==np.array(GT))*100.0/len(Pred))

def ReadLabels(LabelsPathTest, LabelsPathPred):
    if(not (os.path.isfile(LabelsPathTest))):
        print('ERROR: Test Labels do not exist in '+LabelsPathTest)
        sys.exit()
    else:
        LabelTest = open(LabelsPathTest, 'r')
        LabelTest = LabelTest.read()
        LabelTest = map(float, LabelTest.split())

    if(not (os.path.isfile(LabelsPathPred))):
        print('ERROR: Pred Labels do not exist in '+LabelsPathPred)
        sys.exit()
    else:
        LabelPred = open(LabelsPathPred, 'r')
        LabelPred = LabelPred.read()
        LabelPred = map(float, LabelPred.split())
        
    return LabelTest, LabelPred

def ConfusionMatrix(LabelsTrue, LabelsPred):
    """
    LabelsTrue - True labels
    LabelsPred - Predicted labels
    """

    # Get the confusion matrix using sklearn.
    LabelsTrue, LabelsPred = list(LabelsTrue), list(LabelsPred)
    cm = confusion_matrix(y_true=LabelsTrue,  # True class for test-set.
                          y_pred=LabelsPred)  # Predicted class.

    # Print the confusion matrix as text.
    for i in range(10):
        print(str(cm[i, :]) + ' ({0})'.format(i))

    # Print the class-numbers for easy reference.
    class_numbers = [" ({0})".format(i) for i in range(10)]
    print("".join(class_numbers))

    print('Accuracy: '+ str(Accuracy(LabelsPred, LabelsTrue)), '%')
    accuracy = Accuracy(LabelsPred, LabelsTrue)
    return accuracy,cm


ModelPath = "/content/Checkpoints/0a100model.ckpt"
LabelsPath = "/content/data/TxtFiles/LabelsTest.txt"
# transforms = tf.Compose([tf.CenterCrop(10), tf.ToTensor(),tf.RandomRotation((30,70)),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # DenseNet,ResNet (18 layer), ResNet (34 layer)
transforms = tf.Compose([ tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # ResNext
TestSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                    download=True, transform=tf.ToTensor())


# Setup all needed parameters including file reading
ImageSize = SetupAll()

# Define PlaceHolder variables for Predicted output
LabelsPathPred = './TxtFiles/ResNeXtPredOut.txt' # Path to save predicted labels

label_set,pred_set=TestOperation(ImageSize, ModelPath, TestSet, LabelsPathPred)

# Plot Confusion Matrix
LabelsTrue, LabelsPred = ReadLabels(LabelsPath, LabelsPathPred)
Acc, CM = ConfusionMatrix(label_set, pred_set)
sns.heatmap(CM, annot=True, fmt="d")
plt.savefig('ResNeXt.png')
print("Test Accuracy = ", Acc, "%")